In [1]:
import os
import sys
import copy
sys.path.insert(0,'/home/t_goto/hf_env/lib/python3.10/site-packages') # if use virtual environment, add the path of the environment
from datasets import load_from_disk
from utils import InstructDataset, InstructTestDataset, InstructCollator
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch
from torch.utils.data import DataLoader
import copy
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings( #← OpenAIEmbeddingsを初期化する
    model="text-embedding-ada-002"
)

/home/t_goto/hf_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/t_goto/hf_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [2]:
from huggingface_hub import login
token = os.getenv('HF_TOKEN')
login(token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
# load tokenizer
model_name = "jetmoe/jetmoe-8b"
#model_name = "NousResearch/llama-2-7b-chat-hf"
#model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#model_name = "microsoft/phi-1_5"
#model_name = "h2oai/h2o-danube2-1.8b-chat"
#model_name = "medalpaca/medalpaca-7b"
#model_name = "PharMolix/BioMedGPT-LM-7B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [4]:
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    #llm_int8_threshold=200.0,
    #load_in_8bit=True,
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
    #bnb_8bit_use_double_quant=True, # need to avoid cast issue.
    #bnb_8bit_quant_type="nf4",
    #bnb_8bit_compute_dtype=torch.bfloat16,
    #llm_int8_skip_modules= ['decoder', 'lm_head', 'wo'],
)

In [5]:
sys.path.append('/home/t_goto/huggingface/JetMoE') # https://github.com/myshell-ai/JetMoE
from jetmoe import JetMoEForCausalLM, JetMoEConfig, JetMoEForSequenceClassification

In [6]:
#model = AutoModelForCausalLM.from_pretrained(
model = JetMoEForCausalLM.from_pretrained(
    model_name,
    #device_map="auto",
    device_map = {"": 0},
    quantization_config=quant_config
    # torch_dtype=torch.float16, # この時点でtorch.float16を指定すると、train時のlossが0.0になって学習がうまくいかない。原因がよくわかっていません。
)

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.84s/it]


In [7]:
#ft_model_name = '/home/t_goto/huggingface/qlora/jetmoe-8b_FT_med_train'
ft_model_name = '/home/t_goto/huggingface/qlora/llama2_FT_train_adapter1'
#ft_model_name = '/home/t_goto/huggingface/qlora/tiny_llama-1.1b_FT_med_train'
#ft_model_name = '/home/t_goto/huggingface/qlora/phi-1_5_FT_train'
#ft_model_name = "/home/t_goto/huggingface/qlora/h2o-danube2-1.8b_FT_train"
#ft_model=JetMoEForCausalLM.from_pretrained(
ft_model = AutoModelForCausalLM.from_pretrained(
                                        ft_model_name,
                                        device_map = {"": 0},
                                        quantization_config=quant_config)

/home/t_goto/hf_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.38s/it]


In [11]:
data = load_from_disk('meadow_test')
test_dataset = InstructTestDataset(data, tokenizer, ignore_index=tokenizer.pad_token_id) # for use of input_ids (wthout label).
collator = InstructCollator(tokenizer, ignore_index=tokenizer.pad_token_id)
batch_size = 1
test_loader = DataLoader(test_dataset, collate_fn=collator, batch_size=batch_size, shuffle=True)
batch = next(iter(test_loader)) # for checking

100%|███████████████████████████████████████████████████████████████████████████████████████████| 2036/2036 [00:03<00:00, 594.98it/s]


In [8]:
def common_substring(*strings):
    def same_all(*args):
        piv = args[0]
        return all([piv == e for e in args[1:]])
    return ''.join([s[0] for s in itertools.takewhile(lambda x: same_all(*x), zip(*strings))])

In [9]:
from numpy import dot  #← ベクトルの類似度を計算するためにdotをインポート
from numpy.linalg import norm  #← ベクトルの類似度を計算するためにnormをインポート
def comp_vector(ft, annot):
    ft_v = copy.deepcopy(ft)
    annot_v = copy.deepcopy(annot)
    #com_string = common_substring(ft_v, annot_v)
    #ft_v.replace(com_string,'')
    #annot_v.replace(com_string,'')
    q_vector = embeddings.embed_query(annot_v) #← 質問をベクトル化
    ft_vector = embeddings.embed_query(ft_v)
    cos_ft_ann = dot(q_vector, ft_vector) / (norm(q_vector) * norm(ft_vector))
    return cos_ft_ann

In [12]:
# This works fine.
generator = pipeline("text-generation", model=ft_model, tokenizer=tokenizer, max_length=2048, batch_size=8, truncation=True)
cnt = 0
for i, data in enumerate(test_loader):
    #if i>10: break # for checking
    input=tokenizer.decode(data['input_ids'][0].to(0),skip_special_tokens=True)
    out_text=generator(input)
    out_text=out_text[0]['generated_text']

    # extract inference text
    inf_ans = out_text[out_text.find('### Response:')+13:]
    inf_ans = inf_ans.replace('\n',' ')
    #print(f'infer={inf_ans}')
    
    input_text=tokenizer.decode(data['labels'][0],skip_special_tokens=True)
    answer = input_text.replace('\n',' ')
    #print(f'ansr={answer}\n')
    
    similarity = comp_vector(inf_ans, answer)
    if similarity >= 0.9:
        cnt = cnt + 1
    print(f'{i}: {comp_vector(inf_ans, answer)}')
    
print(f'final count = {cnt}')

0: 0.8485443519339644
1: 0.837439088001905
2: 0.8444919862004193
3: 0.9019210244631911
4: 0.8156219291888904
5: 0.8611948310869604
6: 0.8957833068036911
7: 0.8638523937687825
8: 0.8153283774516605


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


9: 0.9072726982041076
10: 0.8349824939445918
11: 0.7976079576797991
12: 0.9581515047658217
13: 0.8838784721836186
14: 0.846015167158106
15: 1.0
16: 0.834746430975319
17: 0.878692336892293
18: 0.882803587042347
19: 0.8203914403350061
20: 0.8553890583898283
21: 0.8672726405347665
22: 0.8873920053774061
23: 0.8991923098825748
24: 0.8645556967884089
25: 0.8254707914927789
26: 0.8792004729243262
27: 0.8639796423522819
28: 0.9073865053257834
29: 0.8416510044337564
30: 0.9045220625722555
31: 0.8869772727640826
32: 0.8678610086293714
33: 0.8472937630260053
34: 0.811711714986955
35: 0.804114253389062
36: 0.8739110731080963
37: 0.7932187589093554
38: 1.0
39: 0.9203368772203753
40: 0.8788373988406886
41: 0.8108269665345266
42: 0.8069565330671206
43: 0.7960452415261937
44: 1.0000000000000002
45: 0.7950899587691869
46: 0.860228751267683
47: 0.8261421361932915
48: 0.8887631173579351
49: 1.0
50: 0.8770711626143586
51: 1.0000000000000002
52: 0.8147133600621742
53: 0.8115415799742491
54: 0.858567946925

In [18]:
model.to(0)

You shouldn't move a model that is dispatched using accelerate hooks.


ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [12]:
batch['input_ids'][0].shape

torch.Size([271])

In [21]:
out_text = tokenizer.decode(outtext[0], skip_special_tokens=True)
inf_ans = out_text[out_text.find('### Response:')+13:]
inf_ans = inf_ans.replace('\n',' ')
print(f'infer={inf_ans}')

infer= You Can Help Others With Migraines If you can relate to the frustration of living with chronic migraine headaches, maybe you can relate to the


In [26]:
tokenizer.decode(batch['labels'][0],skip_special_tokens=True)

'D: Chvostek sign, QT prolongation, decreased PTH, decreased serum calcium, increased serum phosphate'

In [18]:
cnt = 0
for i, data in enumerate(test_loader):
    if i>10: break # for checking
    max_length = len(data['input_ids'][0])
    outputs = model.generate(**data, max_length=max_length+64)
    #out_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    out_text=tokenizer.decode(outputs[0],skip_special_tokens=True)
    
    # extract inference text
    inf_ans = out_text[0][out_text[0].find('### Response:')+13:]
    inf_ans = inf_ans.replace('\n',' ')
    print(f'infer={inf_ans}')
    
    input_text=tokenizer.decode(data['labels'][0],skip_special_tokens=True)
    answer = input_text.replace('\n',' ')
    print(f'ansr={answer}\n')
    
    similarity = comp_vector(inf_ans, answer)
    if similarity >= 0.9:
        cnt = cnt + 1
    print(f'{i}: {comp_vector(inf_ans, answer)}')
    
print(f'final count = {cnt}')

infer=
ansr=Permethrin cream</s>

0: 0.6839361608235051
infer=
ansr=There are several different types of cells within the patch that cannot be differentiated by light microscopy.</s>

1: 0.6778499604948116
infer=
ansr=GABA (gamma-aminobutyric acid)</s>

2: 0.6800058114993698
infer=
ansr=Cervical insufficiency</s>

3: 0.6883423258048216
infer=
ansr=Histological evidence of arteriosclerosis</s>

4: 0.689185970159971
infer=
ansr=t-test</s>

5: 0.6842721106438915
infer=
ansr=Enteral feeding via nasogastric tube</s>

6: 0.6896880501021072
infer=
ansr=Calcium gluconate</s>

7: 0.684354699964441
infer=
ansr=Noncaseating granulomas with multinucleated giant cells</s>

8: 0.6823166597265249
infer=
ansr=D

9: 0.7242770192627866
infer=
ansr=Cardiomegaly and increased bronchial markings</s>

10: 0.6840570224363249
final count = 0


In [20]:
out_text

"Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.\n\n### Instruction:\nPlease answer with one of the option in the bracket\n\n### Input:Q:A 81-year-old man presents to his primary care physician with a 4-month history of shortness of breath. He says that he has slowly lost the ability to do things due to fatigue and now gets winded after walking around the house. He also says that his cough has been getting worse and seems to be producing more sputum. He has gained about 5 pounds over the last 6 months. His past medical history is significant for hypertension and diabetes. He has a 40 pack-year smoking history and drinks about 3 drinks per week. Physical exam reveals a cyanotic appearing man with 1+ edema in his legs bilaterally. He also has wheezing on lung auscultation with a prolonged expiratory phase. Which of the following would most likely be seen on a chest radiograph in t

In [24]:
prompt = "今日は"
max_length = 100
temperature = 0.9
top_p = 0.7

In [26]:
input_ids = tokenizer(prompt, return_tensors="pt")
generated_ids = model.generate(**input_ids, max_length=max_length, temperature=temperature, top_p=top_p)

In [27]:
generated_text = tokenizer.decode(generated_ids[0], bos_token="[BOS]", eos_token="[EOS]")
print(generated_text)

<s> 今日は、「毎日のお昼食」の記事を書いています。

今日は、「毎日のお昼食」の記事を書いています。

今日は、「毎日のお昼食」の記事を書いています。

今日は、「毎日のお


In [28]:
generated_text

'<s> 今日は、「毎日のお昼食」の記事を書いています。\n\n今日は、「毎日のお昼食」の記事を書いています。\n\n今日は、「毎日のお昼食」の記事を書いています。\n\n今日は、「毎日のお'

In [29]:
input_ids

{'input_ids': tensor([[    1, 28705, 30316, 29142, 29277]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}